In [1]:
# Cell 1: Setup & Initialization (Local Environment Version)

import os
from dotenv import load_dotenv
import google.generativeai as genai

# --- Configuration ---
# Load environment variables from the .env file in the root directory
load_dotenv()

# Get the API key from the environment
api_key = os.getenv('GEMINI_API_KEY')

if not api_key:
    print("🔴 ERROR: GEMINI_API_KEY not found in .env file.")
    print("Please create a .env file in the repository root with GEMINI_API_KEY='your-key'.")
else:
    genai.configure(api_key=api_key)

# --- Guild Protocol Functions ---

def load_system_instruction(protocol_file_path="_Admin & Core Docs/Protocols-Frameworks/Copilot_Context_Protocol.md"):
    """Loads the Guild's core protocol document to be used as the system instruction."""
    try:
        # Use os.path.join for cross-platform compatibility
        full_path = os.path.join(os.getcwd(), '..', protocol_file_path) # Assumes notebook is in notebooks/
        if not os.path.exists(full_path):
             full_path = os.path.join(os.getcwd(), protocol_file_path) # Assumes notebook is in root
        
        with open(full_path, 'r', encoding='utf-8') as f:
            print(f"✅ Protocol file found and loaded from: {full_path}")
            return f.read()
    except FileNotFoundError:
        print(f"🔴 ERROR: Protocol file not found.")
        print("Please ensure the path in the 'load_system_instruction' function is correct relative to the repository root.")
        return None

def save_chat_history(chat_session, file_path):
    """Saves the chat history to a JSON file."""
    history = [
        {'role': msg.role, 'parts': [part.text for part in msg.parts]}
        for msg in chat_session.history
    ]
    try:
        with open(file_path, 'w', encoding='utf-8') as f:
            json.dump(history, f, indent=2)
        print(f"✅ Chat history saved to {file_path}")
    except Exception as e:
        print(f"🔴 ERROR saving history: {e}")

def load_chat_history(file_path):
    """Loads a chat history from a JSON file."""
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            return json.load(f)
    except FileNotFoundError:
        print(f"🟡 INFO: No previous history found at {file_path}. Starting a new session.")
        return None
    except Exception as e:
        print(f"🔴 ERROR loading history: {e}")
        return None

def initialize_guild_chat(system_instruction, history=None):
    """Initializes a new chat session with system instruction and optional history."""
    # ... (the generation_config and model setup remains the same) ...
    model = genai.GenerativeModel( ... ) 
    
    # Pass the loaded history to the start_chat method
    chat = model.start_chat(history=history if history else None)
    return chat
    
    generation_config = {
        "temperature": 0.7,
        "top_p": 1,
        "top_k": 32,
        "max_output_tokens": 8192, # Increase the max token output significantly
}
    model = genai.GenerativeModel(
        model_name="gemini-2.5-flash-preview-05-20", # Corrected Model Name
        system_instruction=system_instruction,
        generation_config=generation_config
    )
    chat = model.start_chat()
    return chat

# --- Main Initialization Step ---
print("🛡️  Initializing Guild Interface AI...")
guild_system_protocol = load_system_instruction()

if guild_system_protocol:
    guild_chat_session = initialize_guild_chat(guild_system_protocol)
    if guild_chat_session:
        print("✅  Initialization Complete. AI is calibrated to Guild Protocols.")
        print("➡️  Proceed to run Cell 2 to begin the chat session.")
else:
    print("🔴  Initialization Failed. Please check errors above.")

ModuleNotFoundError: No module named 'dotenv'

In [ ]:
# Cell 2: UI Setup
import ipywidgets as widgets
from IPython.display import display, HTML

# Create the UI components
chat_history_view = widgets.Output(layout={'border': '1px solid black', 'height': '400px', 'overflow_y': 'auto'})
prompt_input = widgets.Text(placeholder='Enter your message...')
send_button = widgets.Button(description='Send')
loading_spinner = widgets.HTML("<p><i>⚫️ [Guild AI]: Compiling response...</i></p>", layout={'display': 'none'})

def display_message(who, message, is_html=False):
    """Appends a message to the chat history view."""
    color = "royalblue" if who == "Operative Kin-Caid" else "green"
    icon = "🔵" if who == "Operative Kin-Caid" else "🟢"
    
    formatted_message = f'<p style="color:{color}; margin-left: 5px;"><strong>{icon} [{who}]:</strong> {message}</p>'
    chat_history_view.append_stdout(formatted_message)

display(HTML("""
<style>
    .widget-text textarea { width: 100% !important; }
    .widget-button button { background-color: #4CAF50 !important; color: white !important; }
</style>
"""))

print("✅  Chat Interface Initialized. Enter your prompt below.")
display(chat_history_view, widgets.HBox([prompt_input, send_button]), loading_spinner)

✅  Chat Session Active. Type 'exit' or 'quit' to end.
--------------------------------------------------


In [ ]:
# Cell 3: UI Logic (The "Backend" for the UI)

def on_send_button_clicked(b):
    prompt = prompt_input.value
    if not prompt.strip():
        return # Do nothing if input is empty
    
    # Display the user's message and show loading spinner
    display_message("Operative Kin-Caid", prompt)
    prompt_input.value = "" # Clear the input box
    loading_spinner.layout.display = 'block'
    
    try:
        # Send message to the AI
        response = guild_chat_session.send_message(prompt)
        
        # Display AI's response (using HTML to render markdown)
        from markdown import markdown
        html_response = markdown(response.text)
        display_message("Guild AI", html_response, is_html=True)

    except Exception as e:
        display_message("SYSTEM", f"<strong>🔴 ERROR:</strong> {e}")
    finally:
        # Hide loading spinner
        loading_spinner.layout.display = 'none'

send_button.on_click(on_send_button_clicked)

Use this to generate chat history

In [ ]:
save_chat_history(guild_chat_session, 'chat_logs/portfolio_project_chat.json')